# LeNet-5 Experiment Reproduction Guide

This guide explains how to reproduce the LeNet-5 (Baseline (fp32), DMC, and Uno) experiments from the "Deep Microcompression" paper.

## Required File Structure

This script assumes it is located within the original project's directory structure under the experiments directory. The development module must be accessible two levels up from this script.

It generates three different models:
- baseline_model: Original model in fp32
- dmc\_ultra: DMC model optimized for size
- dmc\_tiny: DMC model optimized for deployment performance of ATmega32 with a bottleneck of 2KB SRAM.

### Importing the necessary libraries

In [1]:
import sys
import os
import copy
import random

try:
    import torch
    from torch import nn, optim
    from torch.utils import data
    from torchvision import datasets, transforms

except ImportError:
    %pip install torch torchvision tqdm
    import torch
    from torch import nn, optim
    from torch.utils import data
    from torchvision import datasets, transforms

In [2]:
# This assumes the script is in 'project_root/experiments/reproduce_table1'
sys.path.append("../../")

try:
    from development import (
        Sequential,
        Conv2d,
        ConstantPad2d,
        BatchNorm2d,
        ReLU,
        MaxPool2d,
        Flatten,
        Linear,
        EarlyStopper,
        QuantizationGranularity,
        QuantizationScheme
    )
except ImportError:
    print("Error: Could not import the 'development' module.")
    print("Please ensure this script is run from 'experiments/reproduce_table1/'")
    print("and the 'development' module is in the project root ('../../').")


/home/matthias/Documents/EmbeddedAI/deep-microcompression/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Check for a GPU, defaults to the cpu
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# To load a trained model, to skip the initial training step
BASELINE_MODEL_FILE = "lenet5_state_dict.pth"
INPUT_SHAPE = (1, 28, 28)
DATASET_DIR = "../../Datasets"

Using device: cuda


In [4]:
LUCKY_NUMBER = 25

# Set random seed for reproducibility
torch.manual_seed(LUCKY_NUMBER)
if DEVICE == "cuda":
    torch.cuda.manual_seed(LUCKY_NUMBER)
# cuDNN determinism
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
torch.use_deterministic_algorithms(True)


## Trainig Steps

In [5]:
BASE_LINE_TRAIN_EPOCH = 30
DMC_ULTRA_EPOCH = 40
DMC_TINY_EPOCH = 40

## DMC ULTRA COMPRESSION CONFIGURATION

In [ ]:
dmc_ultra_config = {
    'prune_channel': {
        'sparsity': {
            'conv2d_0': 0, 
            'conv2d_1': 9, 
            'linear_0': 64
        },
        'metric': 'l2'
    },
    'quantize': {
        'scheme': QuantizationScheme.STATIC,
        'activation_bitwidth': 8,
        'parameter_bitwidth': {
            'conv2d_0': 4,
            'conv2d_1': 4,
            'linear_0': 4,
            'linear_1': 4
        },
        'granularity': {
            'conv2d_0': QuantizationGranularity.PER_TENSOR,
            'conv2d_1': QuantizationGranularity.PER_TENSOR,
            'linear_0': QuantizationGranularity.PER_TENSOR,
            'linear_1': QuantizationGranularity.PER_TENSOR
        }
    }
}


## DMC TINY COMPRESSION CONFIGURAION


In [ ]:
dmc_tiny_config = {
    'prune_channel': {
        'sparsity': {
            'conv2d_0': 4, 
            'conv2d_1': 2, 
            'linear_0': 0
        },
        'metric': 'l2'
    },
    'quantize': {
        'scheme': QuantizationScheme.STATIC,
        'activation_bitwidth': 4,
        'parameter_bitwidth': {
            'conv2d_0': 8,
            'conv2d_1': 8,
            'linear_0': 4,
            'linear_1': 4
        },
        'granularity': {
            'conv2d_0': QuantizationGranularity.PER_CHANNEL,
            'conv2d_1': QuantizationGranularity.PER_CHANNEL,
            'linear_0': QuantizationGranularity.PER_TENSOR,
            'linear_1': QuantizationGranularity.PER_TENSOR
        }
    }
}

### Getting the MNIST Dataset

In [8]:
# --- Load Data For Training and Testing ---
def get_data_loaders():
    print("Loading MNIST dataset...")
    train_transform = transforms.Compose([
        transforms.RandomRotation(15),
        transforms.RandomAffine(0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    # Test/Inference should NOT have random augments, only normalization
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    mnist_train_dataset = datasets.MNIST(DATASET_DIR, train=True, download=True, transform=train_transform)
    mnist_test_dataset = datasets.MNIST(DATASET_DIR, train=False, download=True, transform=test_transform)
    
    mnist_train_loader = data.DataLoader(mnist_train_dataset, batch_size=32, shuffle=True, num_workers=os.cpu_count(), drop_last=False) # type: ignore
    mnist_test_loader = data.DataLoader(mnist_test_dataset, batch_size=32, shuffle=False, num_workers=os.cpu_count(), drop_last=False) # type: ignore
    
    return mnist_train_loader, mnist_test_loader


In [9]:
# --- Evaluation Functions ---
def accuracy_fun(y_pred, y_true):
    return (y_pred.argmax(dim=1) == y_true).to(torch.float).mean().item() * 100

### Defining and Training the baseline model (Lenet5 fp32)

In [10]:
# --- Define Model ---
def get_baseline_model():
    return Sequential(
        ConstantPad2d(padding=[2]*4, value=0),
        Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, bias=True),
        BatchNorm2d(num_features=6),
        ReLU(),
        MaxPool2d(kernel_size=2, stride=2, padding=0),
        Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, bias=True),
        BatchNorm2d(num_features=16),
        ReLU(),
        MaxPool2d(kernel_size=2, stride=2, padding=0),
        Flatten(),
        Linear(in_features=16*5*5, out_features=84, bias=True),
        ReLU(),
        Linear(in_features=84, out_features=10, bias=True)
    ).to(DEVICE)


## Defining the Training Mechanism

In [11]:
# -------- Trainig the baseline --------------
def train_baseline_model(model, train_loader, test_loader):
    print("\n--- STAGE 1: Training Baseline Model ---")
    if os.path.exists(BASELINE_MODEL_FILE):
        print(f"Loading existing baseline weights from {BASELINE_MODEL_FILE}...")
        model.load_state_dict(torch.load(BASELINE_MODEL_FILE, weights_only=True)["model"], strict=True)
        model.to(DEVICE)
        return model

    print(f"No baseline weights found. Training from scratch (up to 15 epochs)...")
    early_stopper = EarlyStopper(
        monitor_metric="validation_loss",
        delta=1e-7,
        mode="min",
        patience=4,
        restore_best_state_dict=True,
    )
    criterion_fun = nn.CrossEntropyLoss()
    optimizer_fun = optim.Adam(model.parameters(), lr=1.e-3)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_fun, mode="min", patience=2)

    model.fit(
        train_loader, BASE_LINE_TRAIN_EPOCH, 
        criterion_fun, optimizer_fun, lr_scheduler,
        validation_dataloader=test_loader, 
        metrics={"acc": accuracy_fun},
        callbacks=[early_stopper],
        device=DEVICE
    )
    
    print(f"Saving baseline weights to {BASELINE_MODEL_FILE}...")
    torch.save(model.cpu().state_dict(), BASELINE_MODEL_FILE)
    model.to(DEVICE)
    return model

In [12]:
def train_compressed_model(model, compression_config, train_loader, test_loader, epoch, two_step=True):

    def train_step(model, config, epoch, lr, calibration_data):
        compressed_model = model.init_compress(config, INPUT_SHAPE, device=DEVICE, calibration_data=calibration_data)
        criterion_fun = nn.CrossEntropyLoss()
        optimizer_fun = optim.SGD(compressed_model.parameters(), lr=lr, weight_decay=5e-4, momentum=.9)
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_fun, mode="min", patience=1)

        compressed_model.fit(
            train_loader, epoch, 
            criterion_fun, optimizer_fun, lr_scheduler,
            validation_dataloader=test_loader, 
            metrics={"acc": accuracy_fun},
            device=DEVICE
        )
        return compressed_model
    
    # For two step training first train a pruned model and then quantization aware training
    if two_step:
        pruning_config = copy.deepcopy(compression_config)
        del pruning_config["quantize"]

        prune_epoch = int(epoch/2)
        epoch = epoch - prune_epoch

        pruned_model = train_step(model, pruning_config, prune_epoch, lr=1e-3, calibration_data=None)
        model = pruned_model

    calibration_data = next(iter(train_loader))[0].to(DEVICE)
    compressed_model = train_step(model, compression_config, epoch, lr=1e-4, calibration_data=calibration_data)
    
    return compressed_model

## Model Deployment to board

In [ ]:
PROJECT_BASE_DIR = "../../examples/arduino_uno_lenet5"


In [14]:
def save_model_to_board(model, project_base_dir, for_arduino=True):
    src_dir = os.path.join(project_base_dir, "src")
    include_dir = os.path.join(project_base_dir, "include")
    test_image = torch.rand(INPUT_SHAPE, device=DEVICE)

    model = model.fuse(device=DEVICE)
    model.convert_to_c(
        INPUT_SHAPE, "lenet5_model", 
        src_dir, include_dir, 
        for_arduino=for_arduino,
        test_input=test_image
    )
    print(f"Model has been successfully load to {project_base_dir}.")

    model.eval()
    if model.is_quantized and hasattr(model, "output_quantize"):
        print(f"The expected output is {model.output_quantize.apply(model(test_image.unsqueeze(0)))}")
    else:
        print(f"The expected output is {model(test_image.unsqueeze(0))}")


### Baseline Result (fp32)

In [15]:
# Get Data
train_loader, test_loader = get_data_loaders()

# --- STAGE 1: BASELINE ---
baseline_model = get_baseline_model()
baseline_model = train_baseline_model(baseline_model, train_loader, test_loader)

print("Evaluating baseline model...")
baseline_eval = baseline_model.evaluate(test_loader, {"acc": accuracy_fun}, device=DEVICE)
print(f"Baseline Accuracy: {baseline_eval['acc']:.2f}%")


Loading MNIST dataset...

--- STAGE 1: Training Baseline Model ---
Loading existing baseline weights from lenet5_state_dict.pth...
Evaluating baseline model...


Baseline Accuracy: 99.42%


## DMC Best Model

In [16]:
dmc_ultra_model = train_compressed_model(baseline_model, dmc_ultra_config, train_loader, test_loader, DMC_ULTRA_EPOCH)

print("Evaluating baseline model...")
dmc_ultra_eval = dmc_ultra_model.evaluate(test_loader, {"acc": accuracy_fun}, device=DEVICE)
print(f"DMC Ultra Accuracy: {dmc_ultra_eval['acc']:.2f}%")

DMC Training (Epochs 1-20):   5%|▌         | 1/20 [00:27<08:41, 27.47s/it]

epoch    0 | train loss 0.4952 | validation loss 0.0770 | train acc 86.0683 | validation acc 97.8300


DMC Training (Epochs 1-20):  10%|█         | 2/20 [00:59<09:05, 30.33s/it]

epoch    1 | train loss 0.1575 | validation loss 0.0538 | train acc 95.3583 | validation acc 98.2500


DMC Training (Epochs 1-20):  15%|█▌        | 3/20 [01:32<08:52, 31.35s/it]

...

DMC Training (Epochs 1-20):  95%|█████████▌| 19/20 [16:38<00:52, 52.26s/it]

epoch   18 | train loss 0.0913 | validation loss 0.0374 | train acc 97.1683 | validation acc 98.7600


DMC Training (Epochs 1-20): 100%|██████████| 20/20 [17:30<00:00, 52.53s/it]


epoch   19 | train loss 0.0904 | validation loss 0.0380 | train acc 97.1383 | validation acc 98.7700
Evaluating baseline model...


DMC Ultra Accuracy: 98.77%


## Uno Model

In [17]:
dmc_tiny_model = train_compressed_model(baseline_model, dmc_tiny_config, train_loader, test_loader, DMC_TINY_EPOCH)

print("Evaluating baseline model...")
dmc_tiny_eval = dmc_tiny_model.evaluate(test_loader, {"acc": accuracy_fun}, device=DEVICE)
print(f"DMC Tiny Accuracy: {dmc_tiny_eval['acc']:.2f}%")

DMC Training (Epochs 1-20):   0%|          | 0/20 [00:00<?, ?it/s]

DMC Training (Epochs 1-20):   5%|▌         | 1/20 [00:38<12:13, 38.61s/it]

epoch    0 | train loss 0.2125 | validation loss 0.0696 | train acc 93.7800 | validation acc 97.7100


DMC Training (Epochs 1-20):  10%|█         | 2/20 [01:18<11:44, 39.17s/it]

epoch    1 | train loss 0.1209 | validation loss 0.0532 | train acc 96.2767 | validation acc 98.2200


...

DMC Training (Epochs 1-20):  95%|█████████▌| 19/20 [17:18<00:53, 53.26s/it]

epoch   18 | train loss 0.1275 | validation loss 0.0580 | train acc 95.8367 | validation acc 98.2300


DMC Training (Epochs 1-20): 100%|██████████| 20/20 [18:10<00:00, 54.54s/it]


epoch   19 | train loss 0.1274 | validation loss 0.0581 | train acc 95.8117 | validation acc 98.1700
Evaluating baseline model...


DMC Tiny Accuracy: 98.17%


### Final Result

In [18]:
baseline_size = baseline_model.fuse(device=DEVICE).get_size_in_bytes()
dmc_ultra_size = dmc_ultra_model.fuse(device=DEVICE).get_size_in_bytes()
dmc_tiny_size = dmc_tiny_model.fuse(device=DEVICE).get_size_in_bytes()

In [19]:
print("\n--- REPRODUCTION FINISHED ---")
print("\nFinal Results Summary:")
print(f"Baseline:   {baseline_eval['acc']:6.2f}% Acc, {baseline_size/1024:7.2f}KB")
print(f"dmc_ultra:  {dmc_ultra_eval['acc']:6.2f}% Acc, {dmc_ultra_size/1024:7.2f}KB, {dmc_ultra_size/baseline_size*100:.2f}% of original")
print(f"dmc_tiny:   {dmc_tiny_eval['acc']:6.2f}% Acc, {dmc_tiny_size/1024:7.2f}KB, {dmc_tiny_size/baseline_size*100:.2f}% of original")


--- REPRODUCTION FINISHED ---

Final Results Summary:
Baseline:    99.42% Acc,  144.95KB
dmc_ultra:   98.77% Acc,    2.60KB, 1.79% of original
dmc_tiny:    98.17% Acc,   16.05KB, 11.07% of original


## Deplying the model to an Ardunio

In [20]:
model_family = {
    "baseline": baseline_model,
    "dmc_ultra": dmc_ultra_model,
    "dmc_tiny": dmc_tiny_model
}

In [23]:
model_name = "dmc_tiny"
save_model_to_board(model_family[model_name], PROJECT_BASE_DIR, for_arduino=True)


Model has been successfully load to /home/matthias/Documents/EmbeddedAI/deep-microcompression/examples/arduino_uno_lenet5.
The expected output is tensor([[-2,  1, -6,  0, -5, -5, -3, -4, -2, -3]], device='cuda:0',
       dtype=torch.int8)
